# Aluguel

## Pega URL


In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?pagina={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/aluguel/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 1 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)

# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link_aluguel = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link_aluguel)


Progresso: 100%|██████████| 1/1 [00:00<00:00,  4.06it/s]

                                                  URL
0   /imovel/prdio-0-quartos-aluguel-asa-sul-brasil...
1   /imovel/apartamento-2-quartos-aluguel-sul-agua...
2   /imovel/apartamento-2-quartos-aluguel-noroeste...
3   /imovel/sala-0-quartos-aluguel-norte-aguas-cla...
4   /imovel/casa-4-quartos-aluguel-lago-sul-brasil...
5   /imovel/apartamento-3-quartos-aluguel-asa-nort...
6   /imovel/apartamento-1-quarto-aluguel-sudoeste-...
7   /imovel/casa-condominio-4-quartos-aluguel-jard...
8   /imovel/apartamento-4-quartos-aluguel-asa-nort...
9   /imovel/galpo-0-quartos-aluguel-sia-setor-indu...
10  /imovel/kitnet-1-quarto-aluguel-sudoeste-brasi...
11  /imovel/galpo-0-quartos-aluguel-polo-jk-santa-...
12  /imovel/apartamento-3-quartos-aluguel-noroeste...
13  /imovel/galpo-0-quartos-aluguel-saan-setor-ind...
14  /imovel/loja-0-quartos-aluguel-vicente-pires-v...
15  /imovel/casa-4-quartos-aluguel-asa-sul-brasili...
16  /imovel/casa-condominio-4-quartos-aluguel-jard...
17  /imovel/casa-4-quartos-a

## Raspagem dos anúncios

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 mobile-fs-1-5').text.strip()
            preco = soup.find('h6', class_='align-text-left text-normal bb-m mr-6').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            try:
               area = float(area)
            except:
               area = None
            try:
               preco = float(preco)
            except:
               preco = None
            try:
               preco_metro = float(area)/float(preco)
            except:
               preco_metro = None
  
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]

            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": preco,
            "area": area,
            'Valor_metro': preco,
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'aluguel',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link_aluguel['URL'].values]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df_aluguel = pd.DataFrame(info_anuncios_total)

df_aluguel

Extraindo informações dos anúncios: 100%|██████████| 5/5 [00:01<00:00,  3.99it/s]


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.0,7000.0,0,0,0,SemInformacao,2000,ASA SUL,...,0,0,0,0,0,0,0,0,0,Prédio
1,Rua 14,2000.0,63.0,2000.0,2,1,1,86,486,SUL,...,0,0,0,0,0,0,0,0,0,Apartamento
2,SQNW 310 Bloco B,5100.0,76.0,5100.0,2,1,2,409,791,NOROESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
3,Rua COPAIBA,1800.0,32.0,1800.0,0,0,1,SemInformacao,400,NORTE,...,0,0,0,0,0,0,0,0,0,Sala
4,SHIS QL 12,95000.0,1952.0,95000.0,15,0,0,15659,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,0,0,Casa


In [5]:
df_aluguel

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,BRASÍLIA - ASA SUL,200.0,0.028571,0,0,0,SemInformacao,2000,ASA SUL,...,0,0,0,0,0,0,0,0,0,Prédio
1,Rua 14,"Total: 1.800,00 m²",63.0,0.031500,2,1,1,86,486,SUL,...,0,0,0,0,0,0,0,0,0,Apartamento
2,SQNW 310 Bloco B,R$: 409,76.0,0.014902,2,1,2,409,791,NOROESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
3,Rua COPAIBA,"Total: 64.042,00 m²",32.0,0.017778,0,0,1,SemInformacao,400,NORTE,...,0,0,0,0,0,0,0,0,0,Sala
4,SIA Trecho 3,"Total: 8.000,00 m²",6224.0,0.044457,0,0,4,SemInformacao,SemInformacao,SIA,...,0,0,0,0,0,0,0,0,0,Galpão
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,Quadra 102,R$: 199,85.0,0.034000,2,1,1,199,585,NORTE,...,0,1,0,0,0,0,0,0,0,Apartamento
7210,Quadra 201,R$: 65,42.5,0.027419,1,0,1,65,540,SUL,...,0,0,0,0,0,0,0,0,0,Apartamento
7211,Quadra 13 Conjunto G,GAMA - SETOR SUL,40.0,0.043956,1,0,0,SemInformacao,SemInformacao,SETOR SUL,...,0,0,0,0,0,0,0,0,0,Casa
7212,Quadra 13 Conjunto G,GAMA - SETOR SUL,60.0,0.045802,2,0,0,SemInformacao,SemInformacao,SETOR SUL,...,0,0,0,0,0,0,0,0,0,Casa


## Obter Latitude


In [6]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(enderecos):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")

#     coordenadas = []

#     # Itera sobre a lista de endereços com a barra de progresso
#     for endereco in tqdm(enderecos, desc="Progresso"):
#         # Tente encontrar a localização do endereço com um tempo limite maior
#         try:
#             location = geolocator.geocode(endereco, timeout=20)
#         except Exception as e:
#             print(f"Erro ao geocodificar o endereço '{endereco}': {e}")
#             location = None

#         # Se a localização for encontrada, adicione as coordenadas de latitude e longitude à lista
#         if location:
#             coordenadas.append((location.latitude, location.longitude))
#         else:
#             coordenadas.append((None, None))

#         # Espera 1 segundo antes de fazer a próxima solicitação
#         time.sleep(1)

#     return coordenadas

# # Lista de endereços
# enderecos = df_aluguel['Título'].values

# # Chame a função para obter as coordenadas
# coordenadas = obter_coordenadas(enderecos)

# # Crie um DataFrame com os resultados
# df_geo = pd.DataFrame(coordenadas, columns=['Latitude', 'Longitude'])


# df_aluguel = pd.concat([df_aluguel,df_geo],axis=1)

# df_aluguel


In [7]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(endereco):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.geocode(endereco,timeout=10)
#     if location:
#       return location.latitude, location.longitude
#     else:
#       return None, None
# # Lista de endereços
# df_geo_na = df_aluguel[df_aluguel['Latitude'].isna() | df_aluguel['Longitude'].isna()]

# for index, row in tqdm(df_geo_na.iterrows(), desc= 'Obter Coordenadas'):
#   endereco = row['Título'].split()[0]
#   latitude, longitude = obter_coordenadas(endereco)
#   df_aluguel.at[index,'Latitude'] = latitude
#   df_aluguel.at[index,'Longitude'] = longitude


In [8]:
# df[(df['Latitude'] > -15) | (df['Longitude'] < -48)]

## Dados Aluguel

In [9]:
df_aluguel.to_csv('aluguel.csv', index=False)

# Venda

## Pega URL

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?pagina={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/venda/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 20 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)
# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link)

Progresso: 100%|██████████| 20/20 [00:10<00:00,  1.85it/s]

                                                   URL
0    /imovel/lancamento-residencial-wildemir-demart...
1    /imovel/apartamento-4-quartos-venda-sudoeste-b...
2    /imovel/casa-4-quartos-venda-lago-sul-brasilia...
3    /imovel/casa-4-quartos-venda-lago-sul-brasilia...
4    /imovel/lancamento-let-parque-aguas-claras-775541
..                                                 ...
679  /imovel/apartamento-2-quartos-venda-ceilandia-...
680  /imovel/apartamento-3-quartos-venda-asa-norte-...
681  /imovel/apartamento-3-quartos-venda-sudoeste-b...
682  /imovel/apartamento-2-quartos-venda-norte-agua...
683  /imovel/apartamento-4-quartos-venda-noroeste-b...

[684 rows x 1 columns]


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 mobile-fs-1-5').text.strip()
            valor = soup.find('h6', class_='align-text-left text-normal bb-m mr-6').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            try:
               area = float(area)
            except:
               area = None
            try:
               valor = float(valor)
            except:
               valor = None
            try:
               valor_metro = float(area)/float(valor)
            except:
               valor_metro = None
  
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]

            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": valor,
            "area": area,
            'Valor_metro': valor_metro,
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'venda',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link['URL'].values]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df = pd.DataFrame(info_anuncios_total)

df

Extraindo informações dos anúncios:   0%|          | 0/40 [00:00<?, ?it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:  12%|█▎        | 5/40 [00:01<00:08,  4.36it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:  22%|██▎       | 9/40 [00:01<00:06,  4.49it/s]

...

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios:  92%|█████████▎| 37/40 [00:08<00:00,  3.03it/s]

'NoneType' object has no attribute 'text'


Extraindo informações dos anúncios: 100%|██████████| 40/40 [00:08<00:00,  4.60it/s]


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SQSW 300,de Andar do Empreendimento: 6 Andares,147.00,0.000064,4,2,2,SemInformacao,1290,SUDOESTE,...,0,1,0,0,0,1,0,0,0,Apartamento
1,SHIS QL 8,"Total: 1.340,00 m²",750.00,0.000109,5,5,4,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
2,SMDB Conjunto 15,de Andar do Empreendimento: 1 Andar,583.00,0.000119,4,4,3,SemInformacao,1800,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
3,Condomínio Santa Bárbara,"Terreno: 1.018,00 m²",473.95,0.000367,5,3,6,1590,SemInformacao,JARDIM BOTANICO,...,0,0,0,0,0,1,0,0,0,Casa
4,SQS 304,do Imóvel: Frente,177.00,0.000071,3,1,0,SemInformacao,1310,ASA SUL,...,0,0,0,0,0,1,0,0,0,Apartamento
5,SMPW Quadra 26 Conjunto 1,do Imóvel: Vazado,600.00,0.000226,5,4,5,3960,1000,PARK WAY,...,0,1,0,0,0,1,1,0,0,Casa
6,Quadra 206 - Sagitarius Residencial Resort,no Andar: 2,282.00,0.000081,4,4,4,2895,2566,SUL,...,0,1,1,0,0,1,1,0,0,Apartamento
7,SQNW 104 Bloco J,no Andar: 2,141.00,0.000067,3,3,2,4200,1600,NOROESTE,...,0,1,0,0,0,1,0,0,0,Apartamento
8,Quadra 106,"Total: 172,00 m²",172.00,0.000093,3,1,3,SemInformacao,SemInformacao,NORTE,...,0,0,0,0,0,0,0,0,0,Apartamento
9,Condomínio Residencial Santa Mônica,de Andar do Empreendimento: 1 Andar,416.00,0.000220,4,3,4,1900,620,JARDIM BOTANICO,...,0,1,1,0,0,1,1,0,0,Casa


In [14]:
df.to_csv('df_vendas.csv', index=False)

# Obter Latitude

In [15]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(enderecos):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")

#     coordenadas = []

#     # Itera sobre a lista de endereços com a barra de progresso
#     for endereco in tqdm(enderecos, desc="Progresso"):
#         # Tente encontrar a localização do endereço com um tempo limite maior
#         try:
#             location = geolocator.geocode(endereco, timeout=20)  # Aumenta o tempo limite para 10 segundos
#         except Exception as e:
#             print(f"Erro ao geocodificar o endereço '{endereco}': {e}")
#             location = None

#         # Se a localização for encontrada, adicione as coordenadas de latitude e longitude à lista
#         if location:
#             coordenadas.append((location.latitude, location.longitude))
#         else:
#             coordenadas.append((None, None))

#         # Espera 1 segundo antes de fazer a próxima solicitação
#         time.sleep(0)

#     return coordenadas

# # Lista de endereços
# enderecos = df['Título'].values

# # Chame a função para obter as coordenadas
# coordenadas = obter_coordenadas(enderecos)

# # Crie um DataFrame com os resultados
# df_geo = pd.DataFrame(coordenadas, columns=['Latitude', 'Longitude'])


# df = pd.concat([df,df_geo],axis=1)


In [16]:
df['tipo'].value_counts()

tipo
Casa                 16
Apartamento           9
Lote/Terreno/Área     2
Sala                  2
Kitnet                1
Name: count, dtype: int64

In [17]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(endereco):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.geocode(endereco,timeout=10)
#     if location:
#       return location.latitude, location.longitude
#     else:
#       return None, None
# # Lista de endereços
# df_geo_na = df[df['Latitude'].isna() | df['Longitude'].isna()]

# for index, row in tqdm(df_geo_na.iterrows(), desc= 'Obter Coordenadas'):
#   endereco = row['Título'].split()[0]
#   latitude, longitude = obter_coordenadas(endereco)
#   df.at[index,'Latitude'] = latitude
#   df.at[index,'Longitude'] = longitude


In [18]:
df.to_csv('df_venda.csv', index=False)

In [19]:
import pandas as pd

In [20]:
df_aluguel = pd.read_csv('aluguel.csv')
df_venda = pd.read_csv('df_venda.csv')

In [21]:
df_aluguel['tipo'].value_counts()

tipo
Sala                 2020
Apartamento          1860
Loja                 1376
Casa                  897
Prédio                286
Kitnet                286
Galpão                179
Garagem                99
Ponto                  84
Lote/Terreno/Área      75
Hotel                  42
Rural                   9
Loteamento              1
Name: count, dtype: int64

In [22]:
df_venda

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SQSW 300,de Andar do Empreendimento: 6 Andares,147.00,0.000064,4,2,2,SemInformacao,1290,SUDOESTE,...,0,1,0,0,0,1,0,0,0,Apartamento
1,SHIS QL 8,"Total: 1.340,00 m²",750.00,0.000109,5,5,4,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
2,SMDB Conjunto 15,de Andar do Empreendimento: 1 Andar,583.00,0.000119,4,4,3,SemInformacao,1800,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
3,Condomínio Santa Bárbara,"Terreno: 1.018,00 m²",473.95,0.000367,5,3,6,1590,SemInformacao,JARDIM BOTANICO,...,0,0,0,0,0,1,0,0,0,Casa
4,SQS 304,do Imóvel: Frente,177.00,0.000071,3,1,0,SemInformacao,1310,ASA SUL,...,0,0,0,0,0,1,0,0,0,Apartamento
5,SMPW Quadra 26 Conjunto 1,do Imóvel: Vazado,600.00,0.000226,5,4,5,3960,1000,PARK WAY,...,0,1,0,0,0,1,1,0,0,Casa
6,Quadra 206 - Sagitarius Residencial Resort,no Andar: 2,282.00,0.000081,4,4,4,2895,2566,SUL,...,0,1,1,0,0,1,1,0,0,Apartamento
7,SQNW 104 Bloco J,no Andar: 2,141.00,0.000067,3,3,2,4200,1600,NOROESTE,...,0,1,0,0,0,1,0,0,0,Apartamento
8,Quadra 106,"Total: 172,00 m²",172.00,0.000093,3,1,3,SemInformacao,SemInformacao,NORTE,...,0,0,0,0,0,0,0,0,0,Apartamento
9,Condomínio Residencial Santa Mônica,de Andar do Empreendimento: 1 Andar,416.00,0.000220,4,3,4,1900,620,JARDIM BOTANICO,...,0,1,1,0,0,1,1,0,0,Casa


In [23]:
df_total = pd.concat([df_aluguel,df_venda])

In [24]:
df_total.to_csv('imoveis.csv', index=False)


In [25]:
df_total.drop_duplicates()

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,BRASÍLIA - ASA SUL,200.0,0.028571,0,0,0,SemInformacao,2000,ASA SUL,...,0,0,0,0,0,0,0,0,0,Prédio
1,Rua 14,"Total: 1.800,00 m²",63.0,0.031500,2,1,1,86,486,SUL,...,0,0,0,0,0,0,0,0,0,Apartamento
2,SQNW 310 Bloco B,R$: 409,76.0,0.014902,2,1,2,409,791,NOROESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
3,Rua COPAIBA,"Total: 64.042,00 m²",32.0,0.017778,0,0,1,SemInformacao,400,NORTE,...,0,0,0,0,0,0,0,0,0,Sala
4,SIA Trecho 3,"Total: 8.000,00 m²",6224.0,0.044457,0,0,4,SemInformacao,SemInformacao,SIA,...,0,0,0,0,0,0,0,0,0,Galpão
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,SQSW 302 Bloco B - VIA BELVEDERE,do Edifício: VIA BELVEDERE,230.0,0.000062,4,0,3,SemInformacao,SemInformacao,SUDOESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
26,SHA Conjunto 4 Chacará 67,do Imóvel: Frente,300.0,0.000189,3,3,4,SemInformacao,SemInformacao,ARNIQUEIRA,...,0,0,0,0,0,0,0,0,0,Casa
27,Condomínio Residencial Santa Mônica,do Imóvel: Frente,360.0,0.000194,4,3,4,1006,620,JARDIM BOTANICO,...,0,0,0,0,0,1,1,0,0,Casa
28,SRTVN Lote P,"Total: 114,00 m²",50.0,0.000147,0,0,2,3080,836,ASA NORTE,...,0,0,0,0,0,0,0,0,0,Sala
